## Import Section

In [2]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

/Users/radji/miniconda3/envs/dsenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Read Data

Read 2016 Shot Link - Weather data combination and then looking at the sample data, the metadata info, and summary.

In [6]:
df = pd.read_csv('../../golf_course_project_data/combined2012to2016.zip')
df = df.dropna(subset=['StrokesGainedBaseline'])

/Users/radji/miniconda3/envs/dsenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (20,21,22,27,33,34,35,39,41,43,44,45,46,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,TourCode,TourDescription,Year,TournamentNum,PlayerNum,CourseNum,PermanentTournamentNum,PlayerFirstName,PlayerLastName,Round,...,Humidity,Visibility,WindBearing,WindGust,WindSpeed,PrecipitationIntensity,PrecipitationType,CourseName_weather,WeatherDateAndHour,TimeDifference
0,R,PGA TOUR,2012,10,23800,656,16,Bryce,Molder,1,...,0.5,10.0,41.0,nan,9.4,0.0,NaN,Plantation Course at Kapalua,2012-01-06 11:00:00,-1 days +23:35:00.000000000
1,R,PGA TOUR,2012,10,1116,656,16,Michael,Bradley,1,...,0.5,10.0,41.0,nan,9.4,0.0,NaN,Plantation Course at Kapalua,2012-01-06 11:00:00,-1 days +23:36:00.000000000
2,R,PGA TOUR,2012,10,23800,656,16,Bryce,Molder,1,...,0.5,10.0,41.0,nan,9.4,0.0,NaN,Plantation Course at Kapalua,2012-01-06 11:00:00,-1 days +23:41:00.000000000
3,R,PGA TOUR,2012,10,1116,656,16,Michael,Bradley,1,...,0.5,10.0,41.0,nan,9.4,0.0,NaN,Plantation Course at Kapalua,2012-01-06 11:00:00,-1 days +23:42:00.000000000
4,R,PGA TOUR,2012,10,1116,656,16,Michael,Bradley,1,...,0.5,10.0,41.0,nan,9.4,0.0,NaN,Plantation Course at Kapalua,2012-01-06 11:00:00,-1 days +23:45:00.000000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5522327 entries, 0 to 5522326
Data columns (total 65 columns):
TourCode                   object
TourDescription            object
Year                       int64
TournamentNum              int64
PlayerNum                  int64
CourseNum                  int64
PermanentTournamentNum     int64
PlayerFirstName            object
PlayerLastName             object
Round                      int64
TournamentName             object
CourseName_shots           object
Hole                       int64
HoleScore                  float64
ParValue                   int64
Yardage                    int64
Shot                       int64
ShotType                   object
NumStrokes                 int64
FromLocationScorer         object
FromLocationEnhanced       object
ToLocationScorer           object
ToLocationEnhanced         object
Distance                   int64
DistanceToPin              int64
InTheHoleFlag              object
AroundTheGreenF

In [9]:
df.describe()

,Year,TournamentNum,PlayerNum,CourseNum,PermanentTournamentNum,Round,Hole,HoleScore,ParValue,Yardage,...,Hour,Latitude,Longitude,DegreesFahrenheit,Humidity,Visibility,WindBearing,WindGust,WindSpeed,PrecipitationIntensity
count,5522327.0,5522327.0,5522327.0,5522327.0,5522327.0,5522327.0,5522327.0,5521849.0,5522327.0,5522327.0,...,5522327.0,5522327.0,5522327.0,5522327.0,5522327.0,5521234.0,5520552.0,0.0,5520566.0,5522327.0
mean,2014.0,238.1,25636.8,456.6,144.0,2.2,9.5,4.1,4.0,420.1,...,12.8,33.8,-93.8,73.4,0.6,9.5,181.4,nan,7.5,0.0
std,1.4,132.3,8297.3,314.4,204.6,1.1,5.2,0.9,0.6,120.0,...,2.7,5.7,19.4,8.9,0.2,1.1,99.2,nan,3.9,0.0
min,2012.0,10.0,1014.0,4.0,2.0,1.0,1.0,1.0,3.0,104.0,...,0.0,20.7,-157.8,37.0,0.1,1.0,0.0,nan,0.0,0.0
25%,2013.0,130.0,22961.0,24.0,11.0,1.0,5.0,4.0,4.0,386.0,...,11.0,29.9,-98.4,67.6,0.5,9.6,99.0,nan,4.5,0.0
50%,2014.0,230.0,26679.0,609.0,27.0,2.0,10.0,4.0,4.0,442.0,...,13.0,33.6,-86.2,74.8,0.6,9.9,181.0,nan,7.0,0.0
75%,2015.0,340.0,30926.0,729.0,457.0,3.0,14.0,5.0,4.0,490.0,...,15.0,38.2,-80.3,79.8,0.7,10.0,260.0,nan,9.9,0.0
max,2016.0,500.0,50844.0,822.0,518.0,4.0,18.0,13.0,5.0,667.0,...,21.0,45.5,-71.2,99.8,1.0,10.0,359.0,nan,24.0,0.5
